In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("content_topic_prediction/Topic_prediction_data_train.txt",sep='\t',header=None)
data.columns = ['text','uid','type','timestamp']
labels = pd.read_csv("content_topic_prediction/Topic_prediction_labels_train.txt",sep='\t',header=None)
labels.columns = ['label']
data['label'] = labels['label']



In [3]:
data.head()

,text,uid,type,timestamp,label
0,distinct consubstanti translat roman missal pu...,0,Q,0.01,9
1,cathol christian power bind loos catholic beli...,0,Q,0.01,0
2,read scriptur common worship christian church ...,0,Q,0.01,9
3,accord person polit libertarian compat golden ...,0,Q,0.01,0
4,criteria cathol determin testament charact pre...,0,Q,0.01,0


In [ ]:
data.shape

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000)

In [4]:
data = data[(data['text'].notnull())]

In [5]:
data=data.reset_index(drop=True)
data.shape

(96269, 5)

In [ ]:
train_data_features = vectorizer.fit_transform(data["text"])
features= np.array(train_data_features)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
classifier = LogisticRegression(random_state=1)
#classifier= GradientBoostingClassifier()
#classifier= GaussianNB()



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
# scores = cross_val_score(classifier, train_data_features, data["label"], cv=3)
x_Train, x_Test, y_Train,  y_Test= train_test_split(train_data_features,data["label"], test_size=.01)
clf= classifier.fit(x_Train,y_Train)

In [ ]:
text_f=["hello this is a good day","nice day this is", "hello how are you", "hello how are you today", "batman wins all", "hey how are you"]
corpus = pd.DataFrame(columns=["ftext"])
corpus["ftext"] = text_f


In [ ]:
corpus

In [ ]:
kkl=corpus.drop([2])
kkl = kkl.reset_index(drop=True)
kkl

In [ ]:
kkl.iloc[2]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfVect= TfidfVectorizer(min_df=0.0010, ngram_range=[1,3]).fit_transform(data["text"]).todense()

In [ ]:
#tfVect=None

In [7]:
tfVect.shape

(96269, 7578)

In [8]:
from scipy import spatial
import operator


def sim_rank(tx_id,tf_corpus, top=3):
    all_score={}
    
    for i in range(len(tf_corpus)):
        if i==tx_id:
            #print(data.iloc[i]["text"])
            continue
        score= result = 1 - spatial.distance.cosine(tf_corpus[tx_id], tf_corpus[i])
        all_score[i] = score
    sorted_scores = sorted(all_score.items(), key=operator.itemgetter(1))
    sorted_scores.reverse()
    
#     print(data["text"][tx_id])
#     print(data["text"][sorted_scores[0][0]])
#     print(data["text"][sorted_scores[1][0]])
#     print(data["text"][sorted_scores[2][0]])
    return sorted_scores[0:top]
    
        
    
    
    

In [ ]:
all_ranks=sim_rank(353,tfVect,top=50)

In [ ]:
l1=data["text"][353].split(" ")

print(l1, len(l1))

In [ ]:
l2=data["text"][79261].split(" ")
print(l2, len(l2))

In [ ]:
print(list(set(l1).intersection(l2)))
print(len(list(set(l1).intersection(l2))))

In [ ]:
data["text"][15969]

In [ ]:
data.iloc[1]["label"]

In [ ]:
all_ranked_labels=[]
print(all_ranks)
for k in range(len(all_ranks)):
    
    all_ranked_labels.append(data.iloc[all_ranks[k][0]]["label"])
    

In [ ]:
all_ranked_labels

In [11]:
from collections import Counter
print(Counter(all_ranked_labels).keys())
print(Counter(all_ranked_labels).values())

NameError: name 'all_ranked_labels' is not defined

In [ ]:
y

In [ ]:
classifier.predict(x_Test[0])

In [12]:
from collections import Counter
def sim_score(tx_id,tf_corpus, top=3):
    #print("true label - {0}".format(data.iloc[tx_id]["label"]))
    trueL= data.iloc[tx_id]["label"]
    all_ranks=sim_rank(tx_id,tfVect,top=top)
     
    all_ranked_labels=[]
    cumu_scores={}
    for k in range(len(all_ranks)):
        if data.iloc[all_ranks[k][0]]["label"] not in cumu_scores:
            cumu_scores[data.iloc[all_ranks[k][0]]["label"]] = all_ranks[k][1]
        else:
            cumu_scores[data.iloc[all_ranks[k][0]]["label"]] = cumu_scores[data.iloc[all_ranks[k][0]]["label"]] + all_ranks[k][1]
           
        all_ranked_labels.append(data.iloc[all_ranks[k][0]]["label"])
        keys= Counter(all_ranked_labels).keys()
        vals= Counter(all_ranked_labels).values()
   
#     print(all_ranks)
    finalscore= sorted(cumu_scores.items(), key=operator.itemgetter(1))
    finalscore.reverse()
    #print("Predicted label- {0}".format(finalscore[0]))
    #print(cumu_scores, keys, vals) 
    return trueL, finalscore[0][0]
    
    

In [19]:
tl, pl=sim_score(490,tfVect,top=50)

In [20]:
print(tl,pl)

0 0


In [ ]:
# import datetime
# labelt=[]
# labelp=[]
# for i in range(50):
#     print("iteration -{0} at {1}".format(i, datetime.datetime.now()))
#     tl, pl=sim_score(i,tfVect,top=50)
#     labelt.append(tl)
#     labelp.append(pl)
    
    

In [31]:
import random
test_set=random.sample(range(1, 90000), 500)


In [32]:
import random
import datetime
labelt=[]
labelp=[]
countq=0
#Take random ranges of documents to predict.
for i in test_set:
    
    print("iteration -{0} of {1} at {2}".format(countq,i, datetime.datetime.now()))
    tl, pl=sim_score(i,tfVect,top=50)
    labelt.append(tl)
    labelp.append(pl)
    countq +=1

iteration -0 of 29536 at 2018-12-02 06:04:38.736355
iteration -1 of 71740 at 2018-12-02 06:04:45.831394
iteration -2 of 8115 at 2018-12-02 06:04:52.695361
iteration -3 of 14790 at 2018-12-02 06:04:59.525357
iteration -4 of 61618 at 2018-12-02 06:05:06.325394
iteration -5 of 71681 at 2018-12-02 06:05:12.940390
iteration -6 of 2703 at 2018-12-02 06:05:19.583357
iteration -7 of 23711 at 2018-12-02 06:05:26.247399
iteration -8 of 69832 at 2018-12-02 06:05:32.849391
iteration -9 of 576 at 2018-12-02 06:05:39.582393
iteration -10 of 68639 at 2018-12-02 06:05:46.315388
iteration -11 of 7237 at 2018-12-02 06:05:52.964398
iteration -12 of 57986 at 2018-12-02 06:05:59.597359
iteration -13 of 63568 at 2018-12-02 06:06:06.617357
iteration -14 of 30737 at 2018-12-02 06:06:13.357394
iteration -15 of 70786 at 2018-12-02 06:06:20.059397
iteration -16 of 15179 at 2018-12-02 06:06:26.745407
iteration -17 of 27466 at 2018-12-02 06:06:33.473391
iteration -18 of 2053 at 2018-12-02 06:06:40.170368
iteration

iteration -155 of 16285 at 2018-12-02 06:21:58.904716
iteration -156 of 52157 at 2018-12-02 06:22:05.683756
iteration -157 of 50049 at 2018-12-02 06:22:12.396753
iteration -158 of 89662 at 2018-12-02 06:22:19.119753
iteration -159 of 81513 at 2018-12-02 06:22:26.025750
iteration -160 of 856 at 2018-12-02 06:22:32.800769
iteration -161 of 29736 at 2018-12-02 06:22:39.553675
iteration -162 of 5681 at 2018-12-02 06:22:46.312677
iteration -163 of 83992 at 2018-12-02 06:22:53.082677
iteration -164 of 13995 at 2018-12-02 06:22:59.794641
iteration -165 of 26236 at 2018-12-02 06:23:06.593641
iteration -166 of 63099 at 2018-12-02 06:23:13.262641
iteration -167 of 31876 at 2018-12-02 06:23:19.850637
iteration -168 of 65430 at 2018-12-02 06:23:26.574637
iteration -169 of 38219 at 2018-12-02 06:23:33.287641
iteration -170 of 22183 at 2018-12-02 06:23:39.935636
iteration -171 of 26401 at 2018-12-02 06:23:46.627637
iteration -172 of 34312 at 2018-12-02 06:23:53.346640
iteration -173 of 50021 at 2018

iteration -308 of 28583 at 2018-12-02 06:38:59.569477
iteration -309 of 7270 at 2018-12-02 06:39:06.272437
iteration -310 of 50631 at 2018-12-02 06:39:12.863476
iteration -311 of 13543 at 2018-12-02 06:39:19.496477
iteration -312 of 13820 at 2018-12-02 06:39:26.198491
iteration -313 of 86806 at 2018-12-02 06:39:32.855474
iteration -314 of 19327 at 2018-12-02 06:39:39.474476
iteration -315 of 63340 at 2018-12-02 06:39:46.118473
iteration -316 of 88186 at 2018-12-02 06:39:53.352473
iteration -317 of 87155 at 2018-12-02 06:40:00.474477
iteration -318 of 64540 at 2018-12-02 06:40:07.114472
iteration -319 of 59267 at 2018-12-02 06:40:13.693477
iteration -320 of 65277 at 2018-12-02 06:40:20.329436
iteration -321 of 17517 at 2018-12-02 06:40:26.922440
iteration -322 of 73242 at 2018-12-02 06:40:33.596477
iteration -323 of 62715 at 2018-12-02 06:40:40.146472
iteration -324 of 1495 at 2018-12-02 06:40:46.804476
iteration -325 of 75211 at 2018-12-02 06:40:53.497477
iteration -326 of 3729 at 2018

iteration -461 of 14991 at 2018-12-02 06:55:57.916092
iteration -462 of 16403 at 2018-12-02 06:56:04.513087
iteration -463 of 22958 at 2018-12-02 06:56:11.312055
iteration -464 of 89319 at 2018-12-02 06:56:17.929078
iteration -465 of 28476 at 2018-12-02 06:56:24.426088
iteration -466 of 14559 at 2018-12-02 06:56:30.964055
iteration -467 of 39932 at 2018-12-02 06:56:37.552091
iteration -468 of 6364 at 2018-12-02 06:56:44.321329
iteration -469 of 1008 at 2018-12-02 06:56:50.825292
iteration -470 of 66488 at 2018-12-02 06:56:57.473329
iteration -471 of 11163 at 2018-12-02 06:57:04.068324
iteration -472 of 60917 at 2018-12-02 06:57:10.700325
iteration -473 of 89599 at 2018-12-02 06:57:17.355329
iteration -474 of 43068 at 2018-12-02 06:57:23.972324
iteration -475 of 63123 at 2018-12-02 06:57:30.604329
iteration -476 of 30384 at 2018-12-02 06:57:37.248325
iteration -477 of 32431 at 2018-12-02 06:57:43.836324
iteration -478 of 89816 at 2018-12-02 06:57:50.486292
iteration -479 of 24129 at 201

In [33]:
len(labelt)

500

In [34]:
from sklearn.metrics import precision_score, f1_score,recall_score, accuracy_score
accuracy_score(labelt, labelp)

0.396

In [35]:
from sklearn.metrics import classification_report
print(classification_report(labelt, labelp))

              precision    recall  f1-score   support

           0       0.43      0.69      0.53       130
           1       0.33      0.16      0.21        19
           2       0.44      0.34      0.39        32
           3       0.50      0.36      0.42        25
           4       0.41      0.26      0.32        70
           5       0.48      0.33      0.39        49
           6       0.30      0.37      0.33        54
           7       0.62      0.33      0.43        54
           8       0.26      0.21      0.23        38
           9       0.14      0.17      0.16        29

   micro avg       0.40      0.40      0.40       500
   macro avg       0.39      0.32      0.34       500
weighted avg       0.41      0.40      0.38       500

